## Load data from Google Open Images V7

We use fifty one to download only - but we do not use other features of fifty one

In [17]:
# erase folder ˜/fiftyone/open-images-v7
import os
os.system('rm -rf ~/fiftyone/open-images-v7')

0

In [18]:
import os
import fiftyone as fo
import fiftyone.zoo as foz

TargetLabel = "Coffee cup"
TargetImages = 200

try:
    # Load the dataset
    foz.load_zoo_dataset(
        "open-images-v7",
        split="validation",
        label_types=["detections"],
        classes=[TargetLabel],
        max_samples=TargetImages
    )
except Exception as e:
    # if exception is realted to mongo db, it is ok to proceed
    if 'MongoDB' in str(e):
        pass
    else:
        raise e

Only found 155 (<200) samples matching your requirements
 100% |███████████████████| 155/155 [27.2s elapsed, 0s remaining, 6.9 files/s]      
Dataset info written to '/root/fiftyone/open-images-v7/info.json'


In [19]:
# delete all content of ./data folder
os.system('rm -rf ./data')
# images are written into ~/fiftyone/open-images-v7/validation . copy them to ./data
os.system('cp -r ~/fiftyone/open-images-v7/validation ./data')
os.system('ls ./data')


data
labels
metadata


0

In [20]:
# using glob read the names of all files under data/data

import glob
files = glob.glob('./data/data/*')

# create a list called files_names with the names of the files, exlucluding the path and the extension
# hint: use os.path.basename and os.path.splitext
file_names = [os.path.splitext(os.path.basename(f))[0] for f in files]
file_names

['8d279daa12ffc691',
 '48cd906fad6004bf',
 'd44801d0b7da712e',
 'a8b14d4a442bf4a8',
 'dc24d0bfa98cc138',
 '43197f53dc624b26',
 '7facac4942003401',
 '6100e4d81194c50d',
 '60c5d71f33765ee3',
 'ed52f03ad1a8ef84',
 '52072b5c865a4f30',
 'e3847c796a5943b4',
 '5c1caa03e94c6e2e',
 'e6697a9f8ec12c82',
 '079012bc764b139b',
 'a236c800d28a7c66',
 'a32e3afe08e4a238',
 'b0675e458b303b40',
 '780a4baa7c50d6e8',
 'd96609dcb5d94621',
 '8fa6dae079de51d8',
 'c506cdf48321ee2e',
 '34aebaebe39bfaf0',
 'a83d31d69e28914b',
 '40d86cb36246475d',
 '305e993a9b8f4e3d',
 '8ea4b15595293037',
 '9ddd7110b59880c8',
 'ac2a19be6aa6e088',
 'a234b17c1813407c',
 '25d107f1590c1772',
 'f65bae4da0211211',
 '90a35ace03da9909',
 '838de5e8518cde16',
 '282af40d0a1bf7c8',
 '24f774992bd5e264',
 '330b0cc5e49daac6',
 '419d0f32d305cbd0',
 '1172ad179f7d27c6',
 '6c4ad7f5176f72bc',
 'b22b692f610d9810',
 '0b5b9dc7c5f88bff',
 '152adeed60a8a967',
 '31f4daa0d941dcae',
 'd2e270714f830a69',
 '91e09bc4c8db7631',
 '6e72dbc6e751522a',
 '38f856190cd

In [21]:
import pandas as pd

# read data/metadata/classes.csv into dataframe classes - classes does not have headers. first column is called LabelName, second is called LabelDisplayName
classes = pd.read_csv('./data/metadata/classes.csv', header=None, names=['LabelName', 'LabelDisplayName'])

# filter classes to only include the class where LabelName is in df.LabelName
classes = classes[classes['LabelDisplayName'] == TargetLabel]

classes.head()

,LabelName,LabelDisplayName
178,/m/02p5f1q,Coffee cup


In [22]:
# read ./data/labels/detections.csv into a pandas dataframe
import pandas as pd

detect = pd.read_csv('./data/labels/detections.csv')

# keep only rows where LabelName is in classes.LabelName and ImageID is in file_names
detect = detect[detect['LabelName'].isin(classes['LabelName']) & detect['ImageID'].isin(file_names)]
detect


,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
558,0071f62f5d703904,xclick,/m/02p5f1q,1,0.121875,0.345313,0.087683,0.340292,0,0,0,0,0
559,0071f62f5d703904,xclick,/m/02p5f1q,1,0.167187,0.481250,0.392484,0.774530,0,0,0,0,0
560,0071f62f5d703904,xclick,/m/02p5f1q,1,0.368750,0.578125,0.123173,0.409186,0,0,0,0,0
561,0071f62f5d703904,xclick,/m/02p5f1q,1,0.504687,0.854688,0.442589,0.855950,0,0,0,0,0
562,0071f62f5d703904,xclick,/m/02p5f1q,1,0.684375,0.937500,0.175365,0.484342,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296446,f9cdfed65387417e,xclick,/m/02p5f1q,1,0.115523,0.998195,0.097473,0.927798,0,0,0,0,0
301190,fd96bc99468d66c2,xclick,/m/02p5f1q,1,0.000000,0.350000,0.000000,0.314583,0,1,0,0,0
301191,fd96bc99468d66c2,xclick,/m/02p5f1q,1,0.357812,0.848437,0.093750,0.714583,0,0,0,0,1
302049,fe22d79e59610e02,xclick,/m/02p5f1q,1,0.234375,0.579688,0.254167,0.647917,0,0,0,0,0


In [23]:
# add two columns: one with the class name and one with the full file path

# merge detect with classes on LabelName
detect = pd.merge(detect, classes, on='LabelName')

# add a column Class with the value in LabelDisplayName
detect['Class'] = detect['LabelDisplayName']

# add a column Path with the value './data/data/' + ImageID + '.jpg'
detect['Path'] = './data/data/' + detect['ImageID'] + '.jpg'

detect

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,LabelDisplayName,Class,Path
0,0071f62f5d703904,xclick,/m/02p5f1q,1,0.121875,0.345313,0.087683,0.340292,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/0071f62f5d703904.jpg
1,0071f62f5d703904,xclick,/m/02p5f1q,1,0.167187,0.481250,0.392484,0.774530,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/0071f62f5d703904.jpg
2,0071f62f5d703904,xclick,/m/02p5f1q,1,0.368750,0.578125,0.123173,0.409186,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/0071f62f5d703904.jpg
3,0071f62f5d703904,xclick,/m/02p5f1q,1,0.504687,0.854688,0.442589,0.855950,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/0071f62f5d703904.jpg
4,0071f62f5d703904,xclick,/m/02p5f1q,1,0.684375,0.937500,0.175365,0.484342,1,0,0,0,0,Coffee cup,Coffee cup,./data/data/0071f62f5d703904.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,f9cdfed65387417e,xclick,/m/02p5f1q,1,0.115523,0.998195,0.097473,0.927798,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/f9cdfed65387417e.jpg
185,fd96bc99468d66c2,xclick,/m/02p5f1q,1,0.000000,0.350000,0.000000,0.314583,0,1,0,0,0,Coffee cup,Coffee cup,./data/data/fd96bc99468d66c2.jpg
186,fd96bc99468d66c2,xclick,/m/02p5f1q,1,0.357812,0.848437,0.093750,0.714583,0,0,0,0,1,Coffee cup,Coffee cup,./data/data/fd96bc99468d66c2.jpg
187,fe22d79e59610e02,xclick,/m/02p5f1q,1,0.234375,0.579688,0.254167,0.647917,0,0,0,0,0,Coffee cup,Coffee cup,./data/data/fe22d79e59610e02.jpg


In [24]:
# keep only Path, Class, XMin	XMax	YMin	YMax
detect = detect[['Path', 'Class', 'XMin', 'XMax', 'YMin', 'YMax']]
detect

,Path,Class,XMin,XMax,YMin,YMax
0,./data/data/0071f62f5d703904.jpg,Coffee cup,0.121875,0.345313,0.087683,0.340292
1,./data/data/0071f62f5d703904.jpg,Coffee cup,0.167187,0.481250,0.392484,0.774530
2,./data/data/0071f62f5d703904.jpg,Coffee cup,0.368750,0.578125,0.123173,0.409186
3,./data/data/0071f62f5d703904.jpg,Coffee cup,0.504687,0.854688,0.442589,0.855950
4,./data/data/0071f62f5d703904.jpg,Coffee cup,0.684375,0.937500,0.175365,0.484342
...,...,...,...,...,...,...
184,./data/data/f9cdfed65387417e.jpg,Coffee cup,0.115523,0.998195,0.097473,0.927798
185,./data/data/fd96bc99468d66c2.jpg,Coffee cup,0.000000,0.350000,0.000000,0.314583
186,./data/data/fd96bc99468d66c2.jpg,Coffee cup,0.357812,0.848437,0.093750,0.714583
187,./data/data/fe22d79e59610e02.jpg,Coffee cup,0.234375,0.579688,0.254167,0.647917


In [25]:
from collections import defaultdict

# Create a dictionary to hold the bounding boxes for each image
image_bboxes = defaultdict(list)

# Iterate over the rows in the detect dataframe
for _, row in detect.iterrows():
    image_path = row['Path']
    bbox = (row['XMin'], row['XMax'], row['YMin'], row['YMax'])
    image_bboxes[image_path].append(bbox)

# Convert the dictionary to a list of tuples (image_path, bboxes)
merged_bboxes = [(image_path, bboxes) for image_path, bboxes in image_bboxes.items()]
merged_bboxes[0]

('./data/data/0071f62f5d703904.jpg',
 [(0.121875, 0.3453125, 0.08768267, 0.34029227),
  (0.1671875, 0.48125, 0.39248434, 0.7745303),
  (0.36875, 0.578125, 0.123173274, 0.4091858),
  (0.5046875, 0.8546875, 0.44258872, 0.8559499),
  (0.684375, 0.9375, 0.17536534, 0.48434237)])

In [26]:
import shutil

# Rename the folder
os.rename('./data/data', './data/images')

# Remove other folders from data (anything that is not named "images")
for item in os.listdir('./data'):
    item_path = os.path.join('./data', item)
    if item != 'images' and os.path.isdir(item_path):
        shutil.rmtree(item_path)

# List the contents of the data folder to verify
os.listdir('./data')

['images']

In [27]:
# Convert merged_bboxes to a DataFrame
merged_bboxes_df = pd.DataFrame(merged_bboxes, columns=['Path', 'BBoxes'])

# # Save the DataFrame to detections.csv
# merged_bboxes_df.to_csv('./data/detections.csv', index=False)

## Create output in format that can be used for automl for images

In [28]:
import json


def create_annotations(df, output_file):
    # Open the output file in write mode
    with open(output_file, 'w') as f:
        # Iterate over the rows in the merged_bboxes_df dataframe
        for _, row in df.iterrows():
            # Create a dictionary for the current image
            image_dict = {
                "image_url": row['Path'].split('/')[-1],
                # "label": TargetLabel,
                "label": []
            }
            
            # Iterate over the bounding boxes for the current image
            for bbox in row['BBoxes']:
                # Create a dictionary for the current bounding box
                bbox_dict = {
                    "label": TargetLabel,
                    "topX": bbox[0],
                    "topY": bbox[2],
                    "bottomX": bbox[1],
                    "bottomY": bbox[3]
                }
                # Append the bounding box dictionary to the image dictionary
                image_dict["label"].append(bbox_dict)
            
            # Write the image dictionary as a JSON object to the output file
            f.write(json.dumps(image_dict) + '\n')

    # Print the path to the output file
    print(f'JSONL file saved to {output_file}')

In [29]:
# split merged_bbox_df into 3 dataframes: 60% train, 20% validation, 20% test

import numpy as np
train_df, validation_df, test_df = np.split(merged_bboxes_df.sample(frac=1), [int(.6*len(merged_bboxes_df)), int(.8*len(merged_bboxes_df))])



/usr/local/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [30]:
# Save train annotations
create_annotations(train_df, './data/train_annotations.jsonl')

# Save validation annotations
create_annotations(validation_df, './data/validation_annotations.jsonl')

# Save test annotations
create_annotations(test_df, './data/test_annotations.jsonl')

JSONL file saved to ./data/train_annotations.jsonl
JSONL file saved to ./data/validation_annotations.jsonl
JSONL file saved to ./data/test_annotations.jsonl
